In [1]:
import os
import numpy as np
import muspy
from tqdm import tqdm
import config
import utils
import tarfile
import shutil
import collections
from matplotlib import pyplot as plt
import json
import tensorflow as tf
import pickle
import json
import pprint
import subprocess

config_string = "single_instruments_type"
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

conf = config.Config(config_string, ROOT_PATH)

/home/volpepe/miniconda3/envs/music_gen/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-27 19:45:22.479496: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 19:45:22.583500: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-27 19:45:23.067931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open s

In [2]:
# sounfont_path downloaded from https://www.dropbox.com/s/dl/4x27l49kxcwamp5/GeneralUser_GS_1.471.zip
# from the download section of http://schristiancollins.com/generaluser.php
# fluidsynth instead is installed by "brew install fluidsynth" (for mac)

soundfont_path = os.path.join(ROOT_PATH, "data", "GeneralUser GS 1.471", "GeneralUser GS v1.471.sf2")


In [3]:
repr_dir = os.path.join(conf.DATA_PATH, "generated_songs", 'repr')
midi_dir = os.path.join(conf.DATA_PATH, "generated_songs", 'midi')
wav_dir  = os.path.join(conf.DATA_PATH, "generated_songs", 'wav')

os.makedirs(repr_dir, exist_ok=True)
os.makedirs(midi_dir, exist_ok=True)
os.makedirs(wav_dir,  exist_ok=True)

In [4]:
for song_path in os.listdir(repr_dir):
    # Load the song file
    songs = np.load(os.path.join(repr_dir, song_path), allow_pickle=True)
    song_filename = os.path.splitext(song_path)[0]
    # Songs are batched: select them one by one
    for i, song in enumerate(songs):
        # Use anti-transfomation to convert the song to a muspy object
        converted_song  = utils.anti_tranform_representation(song, conf)
        path_song_midi  = os.path.join(midi_dir, f'{song_filename}_{i}.mid')
        # Write midi song
        muspy.outputs.write_midi(
            path=path_song_midi,
            music=converted_song,
            backend="pretty_midi",
        )
        # # Write to wav
        # path_song_wav = os.path.join(wav_dir, f'{song_filename}_{i}.wav')
        # command = ' '.join(["fluidsynth", "-F", path_song_wav, soundfont_path, path_song_midi])
        # subprocess.run(command)